In [1]:
import sys
sys.path.append("../../")

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from utils.kmeans.kmeans import KMeans
from utils.initial_data_handle import filter_genres, handle_non_numerical_data, standarize_dataframe, clean_and_filter_df

sns.set_style("darkgrid")

In [3]:
df = pd.read_csv("../../data/movie_data.csv", delimiter=';', encoding='utf-8')

In [4]:
selected_df, genre_mapping = handle_non_numerical_data(df)
standarized_df = standarize_dataframe(selected_df.dropna())
standarized_df['genres'] = selected_df['genres']
standarized_df.head(3)

,budget,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,vote_average,vote_count,genres
0,0.078947,0.402058,0.038462,0.083333,0.180612,0.176282,0.111111,0.846154,0.384725,1
1,0.171053,0.311705,0.115385,0.083333,0.127060,0.250000,0.222222,0.758242,0.171439,2
2,0.042105,0.070659,0.038462,0.083333,0.039378,0.323718,0.111111,0.670330,0.002416,3


In [5]:
X = standarized_df.to_numpy()

In [6]:
def calculate_sse(max_clusters):
    wcss = []
    for i in range(1, max_clusters+1):
        kmeans = KMeans(i)
        kmeans.fit(X)
        wcss.append(kmeans.compute_sse())

    plt.plot(range(1, max_clusters+1), wcss)
    plt.title('SSE Elbow Method')
    plt.xlabel('Number of Clusters')
    plt.ylabel('SSE')
    plt.show()


In [7]:
# calculate_sse(10)

In [8]:
def calculate_wcss(max_clusters):
    wcss = []
    for i in range(1, max_clusters+1):
        kmeans = KMeans(k=i)
        kmeans.fit(X)
        wcss.append(kmeans.compute_wcss())

    plt.plot(range(1, max_clusters+1), wcss)
    plt.title('WCSS Elbow Method')
    plt.xlabel('Number of Clusters')
    plt.ylabel('WCSS')
    plt.show()


In [9]:
# calculate_wcss(10)

In [10]:
genres_df = filter_genres(df.dropna())
filtered_df, genre_mapping = handle_non_numerical_data(genres_df)
filtered_standarized_df = standarize_dataframe(filtered_df)
X_genre = filtered_standarized_df.to_numpy()

In [11]:
filtered_df.head(3)

,budget,genres,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,vote_average,vote_count
2,16000000.0,1,3.859495,1.0,1.0,81452156.0,127.0,1.0,6.1,34.0
3,60000000.0,2,17.924927,3.0,1.0,187436818.0,170.0,2.0,7.7,1886.0
4,35000000.0,2,5.231580,3.0,1.0,64350171.0,106.0,1.0,5.5,174.0


In [12]:
kmeans = KMeans(k=3)
kmeans.fit(X_genre)

In [13]:
predict_genres_df = filtered_standarized_df.drop(columns='genres', errors='ignore')
X_predict = predict_genres_df.to_numpy()

In [14]:
raw_predictions = kmeans.predict(X_predict)

In [15]:
filtered_df['genres'].unique()

array([1, 2, 3], dtype=int64)

In [16]:
filtered_standarized_df['genres'] = filtered_df['genres'].to_numpy()
filtered_standarized_df['predicted_genres'] = raw_predictions
filtered_standarized_df.head(3)

,budget,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,vote_average,vote_count,genres,predicted_genres
0,0.057143,0.070659,0.038462,0.083333,0.039378,0.260536,0.111111,0.670330,0.002416,1,2
1,0.214286,0.328367,0.115385,0.083333,0.090622,0.425287,0.222222,0.846154,0.133996,2,1
2,0.125000,0.095798,0.115385,0.083333,0.031109,0.180077,0.111111,0.604396,0.012362,2,2


In [17]:
filtered_standarized_df['predicted_genres'].unique()

array([2, 1, 0], dtype=int64)

In [18]:
from utils.initial_data_handle import reverse_genre_mapping


filtered_standarized_df['predicted_genres'] += 1
reverse_genre_mapping(filtered_standarized_df, genre_mapping)
reverse_genre_mapping(filtered_standarized_df, genre_mapping, 'predicted_genres')
filtered_standarized_df.head(3)

,budget,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,vote_average,vote_count,genres,predicted_genres
0,0.057143,0.070659,0.038462,0.083333,0.039378,0.260536,0.111111,0.670330,0.002416,Comedy,Drama
1,0.214286,0.328367,0.115385,0.083333,0.090622,0.425287,0.222222,0.846154,0.133996,Action,Action
2,0.125000,0.095798,0.115385,0.083333,0.031109,0.180077,0.111111,0.604396,0.012362,Action,Drama
